<a href="https://colab.research.google.com/github/altofharsh/colab-xfce/blob/main/colab_xfce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Remote Desktop** : Currently Xfce is used as desktop environment.

Follow the steps below for making your own RDP.
>If you want GPU, you can click on Runtime Tab at top and select Change Runtime Type and Select Hardware accelerator to GPU and save then follow the steps.


# Making User.
> Default username is **user** and password is **root** 



In [ ]:
username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("Setting up!")

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null
    
# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

print("User Created")

# **Installing Desktop Environment and some tools**



In [ ]:
#@markdown  Go to http://remotedesktop.google.com/headless and click **Begin** and then click **Next** then click **Authorise** and copy the **Debian Linux** code and paste below in **CRP**

CRP = "" #@param {type:"string"}

#@markdown Please Enter a Pin you want more or equal to 6 digits (make sure to remember it as you will need it afterwards default pin is **123456**)
Pin = 123456 #@param {type: "integer"}

def CRD():
    with open('install.sh', 'w') as script:
        script.write("""#! /bin/bash

b='\033[1m'
r='\E[31m'
g='\E[32m'
c='\E[36m'
endc='\E[0m'
enda='\033[0m'

printf "\n\n$c$b    Loading Installer $endc$enda" >&2
if sudo apt-get update &> /dev/null
then
    printf "\r$g$b    Installer Loaded $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
    exit
fi

printf "\n$g$b    Installing Chrome Remote Desktop $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb
    sudo dpkg --install chrome-remote-desktop_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Chrome Remote Desktop Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Desktop Environment $endc$enda" >&2
{
    sudo DEBIAN_FRONTEND=noninteractive \
        apt install --assume-yes xfce4 xfce4-goodies
    sudo bash -c 'echo "exec /etc/X11/Xsession /usr/bin/xfce4-session" > /etc/chrome-remote-desktop-session'      sudo apt install --assume-yes xscreensaver
    sudo systemctl disable lightdm.service
} &> /dev/null &&
printf "\r$c$b    Desktop Environment Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Google Chrome $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    sudo dpkg --install google-chrome-stable_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Google Chrome Installed $endc$enda\n" >&2 ||
printf "\r$r$b    Error Occured $endc$enda\n" >&2
sleep 3

printf "$g$b    Installing other Tools $endc$enda" >&2
if sudo apt install nautilus gdebi nano -y &> /dev/null
then
    printf "\r$c$b    Other Tools Installed $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
fi
sleep 3

printf "\n$g$b    Installation Completed $endc$enda\n\n" >&2""")

    ! chmod +x install.sh
    ! ./install.sh

    # Adding user to CRP group
    ! sudo adduser $username chrome-remote-desktop &> /dev/null
    ! sudo update-alternatives --set x-terminal-emulator /usr/bin/xfce4-terminal.wrapper
    # Finishing Work
    ! su - $username -c """$CRP --pin=$Pin""" &> /dev/null

    print("Finished Succesfully")

try:
    if username:
        if CRP == "":
            print("Please enter authcode from the given link")
        elif len(str(Pin)) < 6:
            print("Enter a pin more or equal to 6 digits")
        else:
            CRD()
except NameError:
    print("username variable not found")
    print("Create a User First")





# **Now go to : https://remotedesktop.google.com/access**
> Click on the Online Device and Enter Your Pin !

In [ ]:
#@title **Google Drive Mount**
#@markdown Google Drive can also be used inside the RDP.<br>
#@markdown Mounted at your username's directory which you chose in Step 1. (If `username` variable not defined then use root as default).

def MountGDrive():
    from google.colab import drive

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    mount = """from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)

    with open('/content/mount.py', 'w') as script:
        script.write(mount)

    ! runuser -l $user -c "python3 /content/mount.py"

try:
    if username:
        mountpoint = "/home/"+username+"/drive"
        user = username
except NameError:
    print("username variable not found, mounting at `/content/drive' using `root'")
    mountpoint = '/content/drive'
    user = 'root'

MountGDrive()

# Running without getting disconnected
>Although colab does not encourage hogging resources and I also don't encourage but if you want to run the machine for your projects which takes time you can run this cell.

Also colab has a definate timeout of 12 hours so you cannot run the machine after 12 hours. If you want to run after 12 hours, you can restart the runtime,

In [ ]:
#@title This will prevent you from disconnecting.

while True:pass
